# Лабораторная работа №5: Модель латентных факторов (LFM)

**Сравнение собственной реализации LFM и эталонной реализации Surprise (SVD) на датасете MovieLens 100k**

In [20]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from lfm import LFM


## Загрузка и предобработка данных

In [21]:
def load_movielens_100k(path='data/u.data', max_samples=None):
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    df = pd.read_csv(path, sep='\t', names=names)
    
    # Сначала делаем выборку, если нужно
    if max_samples is not None and len(df) > max_samples:
        df = df.sample(max_samples, random_state=42)
    
    # Затем создаем маппинг для индексации с 0
    user_mapping = {old_id: new_id for new_id, old_id in enumerate(df['user_id'].unique())}
    item_mapping = {old_id: new_id for new_id, old_id in enumerate(df['item_id'].unique())}
    
    df['user_id'] = df['user_id'].map(user_mapping)
    df['item_id'] = df['item_id'].map(item_mapping)
    
    return df

df = load_movielens_100k(max_samples=100000)

n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()
print(f'Пользователей: {n_users}, Фильмов: {n_items}')
print(f'Диапазон user_id: {df["user_id"].min()} - {df["user_id"].max()}')
print(f'Диапазон item_id: {df["item_id"].min()} - {df["item_id"].max()}')
df.head()

Пользователей: 943, Фильмов: 1682
Диапазон user_id: 0 - 942
Диапазон item_id: 0 - 1681


,user_id,item_id,rating,timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596


## Разделение на обучающую и тестовую выборки

In [22]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(f'Обучающая: {len(train)}, Тестовая: {len(test)}')

Обучающая: 80000, Тестовая: 20000


## Обучение собственной реализации LFM

In [24]:
lfm = LFM(n_users, n_items, n_factors=20, lr=0.01, reg=0.01, n_epochs=20)
start = time.time()
lfm.fit(train['user_id'].values, train['item_id'].values, train['rating'].values, verbose=True)
lfm_time = time.time() - start
print(f'Время обучения LFM: {lfm_time:.2f} сек')

rmse_lfm = lfm.rmse(test['user_id'].values, test['item_id'].values, test['rating'].values)
mae_lfm = lfm.mae(test['user_id'].values, test['item_id'].values, test['rating'].values)
print(f'RMSE (LFM): {rmse_lfm:.4f}')
print(f'MAE (LFM): {mae_lfm:.4f}')

Epoch 1/20, RMSE: 2.4284
Epoch 2/20, RMSE: 1.2031
Epoch 3/20, RMSE: 1.0179
Epoch 4/20, RMSE: 0.9554
Epoch 5/20, RMSE: 0.9208
Epoch 6/20, RMSE: 0.8947
Epoch 7/20, RMSE: 0.8721
Epoch 8/20, RMSE: 0.8513
Epoch 9/20, RMSE: 0.8316
Epoch 10/20, RMSE: 0.8126
Epoch 11/20, RMSE: 0.7944
Epoch 12/20, RMSE: 0.7771
Epoch 13/20, RMSE: 0.7609
Epoch 14/20, RMSE: 0.7458
Epoch 15/20, RMSE: 0.7320
Epoch 16/20, RMSE: 0.7192
Epoch 17/20, RMSE: 0.7076
Epoch 18/20, RMSE: 0.6969
Epoch 19/20, RMSE: 0.6872
Epoch 20/20, RMSE: 0.6782
Время обучения LFM: 17.13 сек
RMSE (LFM): 0.9795
MAE (LFM): 0.7633


## Обучение эталонной реализации Surprise (SVD)

In [14]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split as surprise_split
from surprise import accuracy

reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)
trainset, testset = surprise_split(data, test_size=0.2, random_state=42)

algo = SVD(n_factors=20, n_epochs=20, lr_all=0.01, reg_all=0.01, random_state=42)
start = time.time()
algo.fit(trainset)
svd_time = time.time() - start
predictions = algo.test(testset)
rmse_svd = accuracy.rmse(predictions, verbose=True)
mae_svd = accuracy.mae(predictions, verbose=True)
print(f'Время обучения SVD (Surprise): {svd_time:.2f} сек')

RMSE: 0.9576
MAE:  0.7455
Время обучения SVD (Surprise): 0.55 сек


## Сравнение результатов

In [15]:
print('Собственная реализация LFM:')
print(f'  RMSE: {rmse_lfm:.4f}')
print(f'  MAE:  {mae_lfm:.4f}')
print(f'  Время обучения: {lfm_time:.2f} сек')
print('Surprise SVD:')
print(f'  RMSE: {rmse_svd:.4f}')
print(f'  MAE:  {mae_svd:.4f}')
print(f'  Время обучения: {svd_time:.2f} сек')

Собственная реализация LFM:
  RMSE: 0.9795
  MAE:  0.7633
  Время обучения: 16.23 сек
Surprise SVD:
  RMSE: 0.9576
  MAE:  0.7455
  Время обучения: 0.55 сек


In [19]:
movies = pd.read_csv(
    'data/u.item', 
    sep='|', 
    encoding='latin-1', 
    header=None, 
    names=[
        'movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL',
        'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary',
        'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
        'Thriller', 'War', 'Western'
    ]
)

genre = 'War'

genre_movie_ids = set(movies[movies[genre] == 1]['movie_id'] - 1)

users_with_5 = df[(df['item_id'].isin(genre_movie_ids)) & (df['rating'] == 5)]['user_id'].unique()

if len(users_with_5) == 0:
    print(f"Нет пользователей, поставивших 5 фильмам жанра {genre}")
else:
    user_id = users_with_5[0]
    print(f'Пользователь с user_id={user_id} поставил 5 фильмам жанра {genre}')

    # Фильмы, которые пользователь уже смотрел
    watched = set(df[df['user_id'] == user_id]['item_id'])
    not_watched = set(range(n_items)) - watched

    # Предсказываем оценки для всех не просмотренных фильмов
    preds = [(item, lfm.predict_single(user_id, item)) for item in not_watched]
    preds.sort(key=lambda x: x[1], reverse=True)

    top_n = 5
    top_items = [item for item, _ in preds[:top_n]]
    recommended_movies = movies[movies['movie_id'].apply(lambda x: x - 1).isin(top_items)][['title', genre]]
    print(f'Топ-{top_n} рекомендаций для пользователя {user_id}:')
    display(recommended_movies)

Пользователь с user_id=124 поставил 5 фильмам жанра War
Топ-5 рекомендаций для пользователя 124:


,title,War
60,Three Colors: White (1994),0
77,Free Willy (1993),0
280,"River Wild, The (1994)",0
506,"Streetcar Named Desire, A (1951)",0
719,First Knight (1995),0
